<a href="https://colab.research.google.com/github/Gary-Schulke/Project3Startup/blob/master/SQL_queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
import psycopg2
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [0]:
connection = psycopg2.connect(
    host = 'myprojectdb.cqbbih9hnlbh.us-east-1.rds.amazonaws.com',
    port = 5432,
    user = 'root',
    password = 'finalproject',
    database='root'
    )
cursor=connection.cursor()

In [0]:
sql = """
SELECT a.state, a.state_case, a.veh_crash, a.pedestrians, a.motorists_involved, a.rural_urban, a.latitude, a.longitude, \
a.fatalities, a.drunk_drivers, a.city, a.day, a.month, a.year, a.hour, a.light_conditions, d.distract_num, i.imparement_num, \
p.person_num, p.age, p.injury_severity, p.alc_involved, p.alc_test_result, p.drugs, p.drug_num
FROM accident a 
JOIN distract d ON a.state_case = d.state_case AND a.year = d.year
JOIN drimpair i ON a.state_case = i.state_case AND a.year = i.year
JOIN person p ON a.state_case = p.state_case AND a.year = p.year
WHERE a.year = 2012
"""
accident12 = pd.read_sql(sql, con=connection)

In [0]:
sql = """
SELECT a.state, a.state_case, a.veh_crash, a.pedestrians, a.motorists_involved, a.rural_urban, a.latitude, a.longitude, \
a.fatalities, a.drunk_drivers, a.city, a.day, a.month, a.year, a.hour, a.light_conditions, d.distract_num, i.imparement_num, \
p.person_num, p.age, p.injury_severity, p.alc_involved, p.alc_test_result, p.drugs, p.drug_num
FROM accident a 
JOIN distract d ON a.state_case = d.state_case AND a.year = d.year
JOIN drimpair i ON a.state_case = i.state_case AND a.year = i.year
JOIN person p ON a.state_case = p.state_case AND a.year = p.year
WHERE a.year = 2013
"""
accident13 = pd.read_sql(sql, con=connection)

In [0]:
sql = """
SELECT a.state, a.state_case, a.veh_crash, a.pedestrians, a.motorists_involved, a.rural_urban, a.latitude, a.longitude, \
a.fatalities, a.drunk_drivers, a.city, a.day, a.month, a.year, a.hour, a.light_conditions, d.distract_num, i.imparement_num, \
p.person_num, p.age, p.injury_severity, p.alc_involved, p.alc_test_result, p.drugs, p.drug_num
FROM accident a 
JOIN distract d ON a.state_case = d.state_case AND a.year = d.year
JOIN drimpair i ON a.state_case = i.state_case AND a.year = i.year
JOIN person p ON a.state_case = p.state_case AND a.year = p.year
WHERE a.year = 2014
"""
accident14 = pd.read_sql(sql, con=connection)

In [0]:
sql = """
SELECT a.state, a.state_case, a.veh_crash, a.pedestrians, a.motorists_involved, a.rural_urban, a.latitude, a.longitude, \
a.fatalities, a.drunk_drivers, a.city, a.day, a.month, a.year, a.hour, a.light_conditions, d.distract_num, i.imparement_num, \
p.person_num, p.age, p.injury_severity, p.alc_involved, p.alc_test_result, p.drugs, p.drug_num
FROM accident a 
JOIN distract d ON a.state_case = d.state_case AND a.year = d.year
JOIN drimpair i ON a.state_case = i.state_case AND a.year = i.year
JOIN person p ON a.state_case = p.state_case AND a.year = p.year
WHERE a.year = 2015
"""
accident15 = pd.read_sql(sql, con=connection)

In [0]:
sql = """
SELECT a.state, a.state_case, a.veh_crash, a.pedestrians, a.motorists_involved, a.rural_urban, a.latitude, a.longitude, \
a.fatalities, a.drunk_drivers, a.city, a.day, a.month, a.year, a.hour, a.light_conditions, d.distract_num, i.imparement_num, \
p.person_num, p.age, p.injury_severity, p.alc_involved, p.alc_test_result, p.drugs, p.drug_num
FROM accident a 
JOIN distract d ON a.state_case = d.state_case AND a.year = d.year
JOIN drimpair i ON a.state_case = i.state_case AND a.year = i.year
JOIN person p ON a.state_case = p.state_case AND a.year = p.year
WHERE a.year = 2016
"""
accident16 = pd.read_sql(sql, con=connection)

In [0]:
sql = """
SELECT a.state, a.state_case, a.veh_crash, a.pedestrians, a.motorists_involved, a.rural_urban, a.latitude, a.longitude, \
a.fatalities, a.drunk_drivers, a.city, a.day, a.month, a.year, a.hour, a.light_conditions, d.distract_num, i.imparement_num, \
p.person_num, p.age, p.injury_severity, p.alc_involved, p.alc_test_result, p.drugs, p.drug_num
FROM accident a 
JOIN distract d ON a.state_case = d.state_case AND a.year = d.year
JOIN drimpair i ON a.state_case = i.state_case AND a.year = i.year
JOIN person p ON a.state_case = p.state_case AND a.year = p.year
WHERE a.year = 2017
"""
accident17 = pd.read_sql(sql, con=connection)

In [0]:
sql = """
SELECT a.state, a.state_case, a.veh_crash, a.pedestrians, a.motorists_involved, a.rural_urban, a.latitude, a.longitude, \
a.fatalities, a.drunk_drivers, a.city, a.day, a.month, a.year, a.hour, a.light_conditions, d.distract_num, i.imparement_num, \
p.person_num, p.age, p.injury_severity, p.alc_involved, p.alc_test_result, p.drugs, p.drug_num
FROM accident a 
JOIN distract d ON a.state_case = d.state_case AND a.year = d.year
JOIN drimpair i ON a.state_case = i.state_case AND a.year = i.year
JOIN person p ON a.state_case = p.state_case AND a.year = p.year
WHERE a.year = 2018
"""
accident18 = pd.read_sql(sql, con=connection)

In [0]:
accidentdf = pd.concat([accident12, accident13, accident14, accident15, accident16, accident17, accident18])

In [0]:
target = accidentdf["fatalities"]
target_names = [0,1,2,3,4,5]

In [15]:
accidentdf = accidentdf.drop("fatalities", axis=1)
accidentdf = accidentdf.drop("state", axis=1)
accidentdf = accidentdf.drop("state_case", axis=1)
feature_names = accidentdf.columns
accidentdf

,veh_crash,pedestrians,motorists_involved,rural_urban,latitude,longitude,drunk_drivers,city,day,month,year,hour,light_conditions,distract_num,imparement_num,person_num,age,injury_severity,alc_involved,alc_test_result,drugs,drug_num
0,1,0,2,1,32.701317,-85.5252,1,0,1,1,2012,20,2,10,0,1,22,2,0,2,0,1
1,1,0,2,1,32.701317,-85.5252,1,0,1,1,2012,20,2,10,0,1,22,2,0,2,0,1
2,1,0,2,1,32.701317,-85.5252,1,0,1,1,2012,20,2,10,0,2,20,4,8,96,8,0
3,1,0,2,1,32.701317,-85.5252,1,0,1,1,2012,20,2,10,0,2,20,4,8,96,8,0
4,2,0,1,2,34.742508,-86.8835,0,0,1,1,2012,13,1,0,0,1,10,4,0,96,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837471,3,0,3,1,31.747336,-82.4993,0,0,22,10,2018,8,1,96,98,1,22,0,8,996,8,0
837472,1,0,2,2,34.653386,-85.2846,0,0,23,10,2018,16,1,96,0,1,23,4,0,996,8,0
837473,1,0,2,2,34.653386,-85.2846,0,0,23,10,2018,16,1,96,0,1,23,4,0,996,8,0
837474,1,0,2,2,34.653386,-85.2846,0,0,23,10,2018,16,1,96,0,2,0,0,8,996,8,0


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(accidentdf, target, random_state=42)

In [18]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9984873952480223

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.999288998108597

In [20]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.1541137660493383, 'motorists_involved'),
 (0.14910834431899156, 'veh_crash'),
 (0.14595253928470633, 'longitude'),
 (0.10480682488138758, 'latitude'),
 (0.06570230373753343, 'month'),
 (0.06201753442910911, 'day'),
 (0.061724315566221405, 'hour'),
 (0.04830331944886472, 'year'),
 (0.03489541320343597, 'city'),
 (0.026618225551238727, 'rural_urban'),
 (0.02336310347962602, 'light_conditions'),
 (0.021413830170365356, 'age'),
 (0.014835809912219083, 'alc_test_result'),
 (0.01222810727738738, 'distract_num'),
 (0.012093792651567233, 'injury_severity'),
 (0.011367620071256638, 'pedestrians'),
 (0.009391550341385906, 'imparement_num'),
 (0.00903379077111466, 'drugs'),
 (0.008762970912781809, 'drunk_drivers'),
 (0.008516569218114006, 'person_num'),
 (0.008204027573744301, 'drug_num'),
 (0.007546241149610301, 'alc_involved')]

AttributeError: ignored